In [1]:
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score

In [3]:
directory = "mask/"
path = os.listdir(directory)

for i in path:
    
    img = cv2.imread("mask/" + i)
    cv2.imwrite("new_mask/" + i[:-4] + '.png', img)

In [4]:
dir1 = "new_mask/"

dir2 = "results1/"

path = os.listdir(dir1)

path_list = []
    
for i in path:
    
    path_list.append(i)

In [5]:
import os

input_dir = dir1
target_dir = dir2
img_size = (512, 512)
num_classes = 2
batch_size = 4

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".png")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

Number of samples: 20
new_mask/01_test.png | results1/01_test.png
new_mask/02_test.png | results1/02_test.png
new_mask/03_test.png | results1/03_test.png
new_mask/04_test.png | results1/04_test.png
new_mask/05_test.png | results1/05_test.png
new_mask/06_test.png | results1/06_test.png
new_mask/07_test.png | results1/07_test.png
new_mask/08_test.png | results1/08_test.png
new_mask/09_test.png | results1/09_test.png
new_mask/10_test.png | results1/10_test.png


In [6]:
# Import the necessary libraries
from PIL import Image
import numpy as np

# load the image and convert into numpy array

img1 = Image.open(dir1 + "01_test.png")
img2 = Image.open(dir2 + "01_test.png")
  
# asarray() class is used to convert
# PIL images into NumPy arrays
numpy_data1 = np.asarray(img1)
numpy_data2 = np.asarray(img2)

# Consider only the first two channels
numpy_data1 = numpy_data1[:,:,0] # Discard the last channel 
numpy_data2 = numpy_data2[:,:,0] # Discard the last channel 

# Alternative
# numpy_data1 = numpy_data1[:,:,0] and then numpy_data1 = numpy_data1[:,:,1]
  
# <class 'numpy.ndarray'>
print(type(numpy_data1))
  
#  shape
print(numpy_data1.shape)

<class 'numpy.ndarray'>
(512, 512)


In [7]:
data1 = []
data2 = []

for i in range(20):
    img1 = Image.open(dir1 + path_list[i])
    img2 = Image.open(dir2 + path_list[i])
    
    np_data1 = np.asarray(img1)
    np_data2 = np.asarray(img2)
    
    # Consider only the first two channels
    np_data1 = np_data1[:,:,0] # Discard the last channel 
    np_data2 = np_data2[:,:,0] # Discard the last channel 
    
    data1.append(np_data1)
    data2.append(np_data2)
    
data1 = np.array(data1)
data2 = np.array(data2)

print(data1.shape)
print(data2.shape)

(20, 512, 512)
(20, 512, 512)


In [8]:
print(np.array_equal(numpy_data1, numpy_data2))

False


In [9]:
print(np.array_equiv(numpy_data1, numpy_data2))

False


In [10]:
final_shape = numpy_data1.shape[0]*numpy_data1.shape[1]

a = np.reshape(numpy_data1, final_shape)
b = np.reshape(numpy_data2, final_shape)

y_score = np.empty((a.shape[0]))

for i in range(a.shape[0]):
    if a[i] == b[i]:
        y_score[i] = 1
    else:
        y_score[i] = 0
        
y_score

array([1., 1., 1., ..., 1., 1., 1.])

In [11]:
threshold_value = 0.5
y_true = np.empty((a.shape[0]))

for i in range(a.shape[0]):
    if np.float32(int(a[i]))/255.0 > 0.5:
        y_true[i] = 0
    else:
        y_true[i] = 1
        
y_true

array([1., 1., 1., ..., 1., 1., 1.])

In [12]:
print("The Confusion Matrix = ")
confusion_matrix(y_true, y_score)

The Confusion Matrix = 


array([[     0,      0],
       [ 62069, 200075]], dtype=int64)

In [13]:
confusion = confusion_matrix(y_true, y_score)

In [14]:
# confusion = confusion_matrix(y_true, y_score)

specificity = 0
if float(confusion[0,0]+confusion[0,1])!=0:
    specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
# print("Specificity: " +str(specificity))

sensitivity = 0
if float(confusion[1,1]+confusion[1,0])!=0:
    sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
print("Sensitivity: " +str(sensitivity))

precision = 0
if float(confusion[1,1]+confusion[0,1])!=0:
    precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
    
print("Precision: " +str(precision))

# F1 score
F1_score = f1_score(y_true, y_score, labels=None, average='binary', sample_weight=None)
print("\nF1 score (F-measure): " +str(F1_score))

Sensitivity: 0.7632255554199219
Precision: 1.0

F1 score (F-measure): 0.8657151696490192


In [15]:
# Compute The IoU Scores:

intersection_values = 0
union_values = 0

for i, j in zip(y_true, y_score):
    if i and j == 1:
        intersection_values += 1
        
for i, j in zip(y_true, y_score):
    if i or j == 1:
        union_values += 1

In [16]:
print("The IoU Score for the single image = ", np.float32(intersection_values/union_values))

The IoU Score for the single image =  0.76322556


## Individual Blocks:

In [17]:
import pandas as pd

df = pd.DataFrame(columns = ['Recall', 'Precision', 'Accuracy', 'F1-Score'])

In [18]:
count = 0

for i, j in zip(os.listdir(dir1), os.listdir(dir2)):
    
    append_list = []
    
    img1 = Image.open(dir1 + i)
    img2 = Image.open(dir2 + j)

    numpy_data1 = np.asarray(img1)
    numpy_data2 = np.asarray(img2)
#     print(numpy_data1.shape)
    
    # Consider only the first two channels
    numpy_data1 = numpy_data1[:,:,0] # Discard the last channel 
    numpy_data2 = numpy_data2[:,:,0] # Discard the last channel 
#     print(numpy_data1.shape)
    
    final_shape = numpy_data1.shape[0]*numpy_data1.shape[1]
#     print(final_shape)

    a = np.reshape(numpy_data1, final_shape)
    b = np.reshape(numpy_data2, final_shape)

    y_score = np.empty((a.shape[0]))

    for i in range(a.shape[0]):
        if a[i] == b[i]:
            y_score[i] = 1
        else:
            y_score[i] = 0

    threshold_value = 0.5
    y_true = np.empty((a.shape[0]))

    for i in range(a.shape[0]):
        if np.float32(int(a[i])-int(b[i]))/255.0 > 0.5:
            y_true[i] = 0
        else:
            y_true[i] = 1
            
    count += 1
    
    print(f"\n Image{count}")
            
#     print("The Confusion Matrix = ")
#     print(confusion_matrix(y_true, y_score))
    
    confusion = confusion_matrix(y_true, y_score)
    
    # confusion = confusion_matrix(y_true, y_score)

    specificity = 0
    if float(confusion[0,0]+confusion[0,1])!=0:
        specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
#     print("Specificity: " +str(specificity))

    sensitivity = 0
    if float(confusion[1,1]+confusion[1,0])!=0:
        sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
    print("Sensitivity: " +str(sensitivity))
    append_list.append(sensitivity)
    
    precision = 0
    if float(confusion[1,1]+confusion[0,1])!=0:
        precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])

    print("Precision: " +str(precision))
    append_list.append(precision)
    
    # Accuracy
    
    accuracy = 0

    accuracy = float(confusion[0,0]+confusion[1,1])/float(confusion[0,0]+confusion[0,1]+confusion[1,0]+confusion[1,1])
    print("Accuracy: " +str(accuracy))
    append_list.append(accuracy)

    # F1 score
    F1_score = f1_score(y_true, y_score, labels=None, average='binary', sample_weight=None)
    print("F1 score (F-measure): " +str(F1_score))
    append_list.append(F1_score)
    
    df.loc[len(df)] = append_list
        
#     if count == 10:
#         break


 Image1
Sensitivity: 0.7632255554199219
Precision: 1.0
Accuracy: 0.7632255554199219
F1 score (F-measure): 0.8657151696490192

 Image2
Sensitivity: 0.7551689147949219
Precision: 1.0
Accuracy: 0.7551689147949219
F1 score (F-measure): 0.8605085338845094

 Image3
Sensitivity: 0.7672080993652344
Precision: 1.0
Accuracy: 0.7672080993652344
F1 score (F-measure): 0.8682713706900832

 Image4
Sensitivity: 0.7722740173339844
Precision: 1.0
Accuracy: 0.7722740173339844
F1 score (F-measure): 0.8715063356802004

 Image5
Sensitivity: 0.7597885131835938
Precision: 1.0
Accuracy: 0.7597885131835938
F1 score (F-measure): 0.8634997984037042

 Image6
Sensitivity: 0.7586936950683594
Precision: 1.0
Accuracy: 0.7586936950683594
F1 score (F-measure): 0.8627923068080021

 Image7
Sensitivity: 0.7643318176269531
Precision: 1.0
Accuracy: 0.7643318176269531
F1 score (F-measure): 0.8664263830541676

 Image8
Sensitivity: 0.7823066711425781
Precision: 1.0
Accuracy: 0.7823066711425781
F1 score (F-measure): 0.877858657

In [19]:
df.to_csv('values-0.csv')

## GLOBAL:

In [20]:
final_shape = data1.shape[0]*data1.shape[1]*data1.shape[2]

c = np.reshape(data1, final_shape)
d = np.reshape(data2, final_shape)

y_score_all = np.empty((c.shape[0]))

for i in range(c.shape[0]):
    if c[i] == d[i]:
        y_score_all[i] = 1
    else:
        y_score_all[i] = 0
        
y_score_all

array([1., 1., 1., ..., 1., 1., 1.])

In [21]:
threshold_value = 0.5
y_true_all = np.empty((c.shape[0]))

for i in range(c.shape[0]):
    if np.float32(int(c[i]))/255.0 > 0.5:
        y_true_all[i] = 0
    else:
        y_true_all[i] = 1
        
y_true_all

array([1., 1., 1., ..., 1., 1., 1.])

In [22]:
print("The Global Confusion Matrix = ")
confusion_matrix(y_true_all, y_score_all)

The Global Confusion Matrix = 


array([[      0,       0],
       [1198620, 4044260]], dtype=int64)

In [23]:
confusion = confusion_matrix(y_true_all, y_score_all)

In [24]:
# confusion = confusion_matrix(y_true, y_score)

specificity = 0
if float(confusion[0,0]+confusion[0,1])!=0:
    specificity = float(confusion[0,0])/float(confusion[0,0]+confusion[0,1])
# print("Specificity: " +str(specificity))

sensitivity = 0
if float(confusion[1,1]+confusion[1,0])!=0:
    sensitivity = float(confusion[1,1])/float(confusion[1,1]+confusion[1,0])
print("Sensitivity: " +str(sensitivity))

precision = 0
if float(confusion[1,1]+confusion[0,1])!=0:
    precision = float(confusion[1,1])/float(confusion[1,1]+confusion[0,1])
    
print("Precision: " +str(precision))

# F1 score
F1_score = f1_score(y_true, y_score, labels=None, average='binary', sample_weight=None)
print("\nF1 score (F-measure): " +str(F1_score))

Sensitivity: 0.7713813781738281
Precision: 1.0

F1 score (F-measure): 0.8907784192966236


In [25]:
accuracy = 0

accuracy = float(confusion[1,1]+confusion[0,0])/float(confusion[0,0]+confusion[0,1]+confusion[1,0]+confusion[1,1])
print("Accuracy: " +str(accuracy))

Accuracy: 0.7713813781738281


In [26]:
# Compute The IoU Scores:

intersection_values = 0
union_values = 0

for i, j in zip(y_true_all, y_score_all):
    if i and j == 1:
        intersection_values += 1
        
for i, j in zip(y_true_all, y_score_all):
    if i or j == 1:
        union_values += 1
        
print("The IoU Score for the images = ", np.float32(intersection_values/union_values))

The IoU Score for the images =  0.7713814
